In [1]:
import cv2
import time
import numpy as np

In [2]:
# Tracking işlemi yapıldığı için RAM dolup kod kapanmasın diye bunu çalıştırıyoruz.
# Bazı donanım ya da kütüphane sürümlerine gerekli olmayabilir.
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
from ultralytics import YOLO
model = YOLO("yolov8l.pt")

100%|██████████| 83.7M/83.7M [00:19<00:00, 4.39MB/s]


In [4]:
font = cv2.FONT_HERSHEY_DUPLEX
kamera = cv2.VideoCapture("video.mp4")



region1=np.array([(500,0),(500,720),(550,720),(550,0)])
region1 = region1.reshape((-1,1,2))

total=set()
while True:
    
    

    ret, frame = kamera.read()
    
    if not ret:
        break
        
        
    rgb_img=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    results = model.track(rgb_img, persist=True, verbose=False)
    labels=results[0].names
 
    cv2.line(frame, (550,0), (550,720), (0,0,255), 10)
   
    for i in range(len(results[0].boxes)):
        x1,y1,x2,y2=results[0].boxes.xyxy[i]
        score=results[0].boxes.conf[i]
        cls=results[0].boxes.cls[i]
        ids=results[0].boxes.id[i]
    
        x1,y1,x2,y2,score,cls,ids=int(x1),int(y1),int(x2),int(y2),float(score),int(cls),int(ids)
        name=labels[cls]
        

        
            
        if score<0.5:
            continue
        if name!='sheep':
            continue
        cx=int(x1/2+x2/2)
        cy=int(y1/2+y2/2)
        cv2.circle(frame,(cx,cy),10,(255,0,0),-1)
        
        inside_region1=cv2.pointPolygonTest(region1,(cx,cy),False)
        if inside_region1>0:
            total.add(ids)
        
    
 
    total_str='TOTAL: '+str(len(total))
    
    frame[0:70,0:360]=(102,0,153)
    
    cv2.putText(frame, total_str,(0, 60), font, 2.2, (255,255,255), 2)


    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
kamera.release()
cv2.destroyAllWindows()